# Cross-Validation y Predicción con YOLO11 Pose


In [5]:
from ultralytics import YOLO
import cv2
import os
import yaml
import numpy as np

MODEL_NAME = "yolo11x-pose.pt" 
K_FOLDS = 5
IMG_SIZE = 640
BATCH_SIZE = 6

BASE_IMG_DIR = "images"
BASE_LBL_DIR = "labels"
TEST_DIR = "images/test"

OUT_LABEL_TEST = "labels/test_pred"
OUT_IMAGE_TEST = "predictions_test"

os.makedirs(OUT_LABEL_TEST, exist_ok=True)
os.makedirs(OUT_IMAGE_TEST, exist_ok=True)

## Función para generar YAML dinámico
Cada fold crea su propio archivo YAML con rutas absolutas.

In [6]:
def crear_yaml_fold(fold_idx, train_folds, val_fold):
    cwd = os.getcwd()

    train_paths = [os.path.join(cwd, BASE_IMG_DIR, f) for f in train_folds]
    val_path = os.path.join(cwd, BASE_IMG_DIR, val_fold)

    yaml_data = {
        'path': cwd,
        'train': train_paths,
        'val': val_path,
        'names': {0: 'salmon'},
        'kpt_shape': [4, 3]
    }

    yaml_filename = f"data_fold_{fold_idx}.yaml"
    with open(yaml_filename, 'w') as f:
        yaml.dump(yaml_data, f, sort_keys=False)

    return yaml_filename

## Cross Validation
Se entrena un modelo distinto por cada fold y se selecciona el mejor a partir del mAP50-95.

In [7]:
def cross_validation():
    print("\nINICIANDO CROSS-VALIDATION...\n")

    fold_metrics = []
    best_map = 0
    best_model_path = ""

    all_folds = [f"fold{i}" for i in range(1, K_FOLDS + 1)]

    for i in range(K_FOLDS):
        current_fold_idx = i + 1

        val_fold = all_folds[i]
        train_folds = [f for j, f in enumerate(all_folds) if j != i]

        print(f"\n===== FOLD {current_fold_idx}/{K_FOLDS} =====")
        print(f"Train: {train_folds}")
        print(f"Val:   {val_fold}\n")

        yaml_file = crear_yaml_fold(current_fold_idx, train_folds, val_fold)

        model = YOLO(MODEL_NAME)

        project_name = "runs/cv_pose"
        run_name = f"fold_{current_fold_idx}"

        model.train(
            data=yaml_file,
            imgsz=IMG_SIZE,
            epochs=20,
            batch=BATCH_SIZE,
            device=0,
            project=project_name,
            name=run_name,
            exist_ok=True,
            plots=True,
            verbose=False
        )

        metrics = model.val()
        map50_95 = metrics.pose.map

        print(f"Resultados Fold {current_fold_idx}: mAP50-95 = {map50_95:.4f}")
        fold_metrics.append(map50_95)

        if map50_95 > best_map:
            best_map = map50_95
            best_model_path = os.path.join(project_name, run_name, "weights", "best.pt")

    print("\nRESULTADOS CROSS-VALIDATION")
    for i, m in enumerate(fold_metrics):
        print(f"Fold {i+1}: {m:.4f}")

    print("--------------------------")
    print(f"PROMEDIO mAP: {np.mean(fold_metrics):.4f}")
    print(f"MEJOR MODELO: {best_model_path}")

    return best_model_path

## Predicción en conjunto de test
Se generan etiquetas automáticas y una visualización por cada imagen.

In [8]:
def predecir_test(model_path):
    print(f"\nGENERANDO ETIQUETAS PARA TEST ({TEST_DIR})...\n")

    if not os.path.exists(model_path):
        print("Error: No se encuentra el mejor modelo.")
        return

    model = YOLO(model_path)
    imgs = [f for f in os.listdir(TEST_DIR) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

    print(f"Procesando {len(imgs)} imágenes de test...")

    for i, name in enumerate(imgs):
        img_path = os.path.join(TEST_DIR, name)

        results = model(img_path, device=0, conf=0.25, verbose=False)[0]

        if results.keypoints is None:
            continue

        kpts_norm = results.keypoints.xyn.cpu().numpy()
        if len(kpts_norm) == 0:
            continue

        txt_path = os.path.join(OUT_LABEL_TEST, name.rsplit(".", 1)[0] + ".txt")
        with open(txt_path, "w") as f:
            for kpts in kpts_norm:
                linea = "0 0.5 0.5 1.0 1.0 "
                for kx, ky in kpts:
                    vis = 2 if (kx > 0 or ky > 0) else 0
                    linea += f"{kx:.6f} {ky:.6f} {vis} "
                f.write(linea.strip() + "\n")

        img = cv2.imread(img_path)
        if img is not None:
            kpts_abs = results.keypoints.xy.cpu().numpy()
            for person in kpts_abs:
                for x, y in person:
                    if x == 0 and y == 0:
                        continue
                    cv2.circle(img, (int(x), int(y)), 4, (0, 255, 0), -1)
            cv2.imwrite(os.path.join(OUT_IMAGE_TEST, f"pred_{name}"), img)

        if i % 10 == 0:
            print(f"Progreso test: {i}/{len(imgs)}...")

    print("\nProceso completado.")
    print(f"Etiquetas TEST generadas en: {OUT_LABEL_TEST}")

## Ejecutar Cross Validation y Predicción Final

In [9]:
mejor_modelo = cross_validation()
predecir_test(mejor_modelo)


INICIANDO CROSS-VALIDATION...


===== FOLD 1/5 =====
Train: ['fold2', 'fold3', 'fold4', 'fold5']
Val:   fold1

New https://pypi.org/project/ultralytics/8.3.233 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.221  Python-3.12.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data_fold_1.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train,